In [1]:
from pattern.en import lemma
from DependencyGraphHandler import DependencyGraphHandler
d = DependencyGraphHandler()

Created file: /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/preliminary/parameters.json
Parameters(
  base_dir = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon
  data_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/data/parsed/five-three_5995.json
  lexicon_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/data/parsed/lexicon_6788.json
  output_dir = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/preliminary
  output_sub_dir = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/preliminary/sub
  parameters_json_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/preliminary/parameters.json
  output_time_txt_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/preliminary/elapsed_time.txt
  output_report_csv_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/preliminary/report.csv
)


In [2]:
from config import parameters
import pandas as pd
from collections import defaultdict
df = pd.read_json(parameters.data_filepath)

# Target preprocessing

In [3]:
def process_targets(content, targets):
    processed_targets = []
    for target in targets:
        for token in content.split():
            if target in token: processed_targets.append(token)
    return list(set(processed_targets))

df['targets'] = df.apply(lambda x: process_targets(x['content'], x['target']), axis=1)

In [4]:
df.head()

,content,target,filename,targets
0,"this is an edited review , now that i have had...",[],five-Creative Labs Nomad Jukebox Zen Xtra 40GB,[]
1,"while , there are flaws with the machine , the...",[affordability],five-Creative Labs Nomad Jukebox Zen Xtra 40GB,[affordability]
10,"for me , i 'll save that cash for another purc...",[],five-Creative Labs Nomad Jukebox Zen Xtra 40GB,[]
100,"the face-plate , which pops out for the batter...",[face plate],five-Creative Labs Nomad Jukebox Zen Xtra 40GB,[]
1000,after i reviewed the manual ( a pdf file inclu...,[],five-Creative Labs Nomad Jukebox Zen Xtra 40GB,[]


In [5]:
opinion_word_lexicon = [item for sublist in pd.read_json(parameters.lexicon_filepath).values for item in sublist]
def match_opinion_words(content):
    opinion_words = []
    for opinion in opinion_word_lexicon:
        for token in content.split():
            if token == opinion: opinion_words.append(token)
    return list(set(opinion_words))
df['opinion_words'] = df.apply(lambda x: match_opinion_words(x['content']), axis=1)

Outlier detector

In [6]:
# def contain_targets(content, targets):
#     for target in targets:
#         if target not in content: return True
#     return False
# df['outlier'] = df.apply(lambda x: contain_targets(x['content'], x['target']), axis=1)

# import copy
# original_df = copy.deepcopy(df)

In [7]:
# original_df[original_df.index==90]['content'].values

In [8]:
# original_df[original_df['outlier']==True]

In [9]:
# outlier_sentence_index = df[df['outlier']==True].index
# print('# of outliers not containing the targets in a sentence:', len(outlier_sentence_index))
# df = df.drop(outlier_sentence_index)

# print("[AFTER] # of contents = %d" % len(df))

In [10]:
# temp
# df[df['content']=='a good player at a great price with terrible software that makes ripping and transferring way more difficult than it should be .']

In [11]:
import stanfordnlp
nlp = stanfordnlp.Pipeline()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt.pr

In [12]:
df_temp = df.iloc[:50]
pattern_counter = defaultdict(int)

err_list = []
for _, row in df_temp.iterrows():
    document = row['content']
    doc = nlp(document)
    for sentence_from_doc in doc.sentences:
        o_t = [(o,t) for o in row['opinion_words'] for t in row['targets']]
        for o_word,t_word in o_t:
            flattened_string = ''.join([sentence_from_doc.words[i].text for i in range(len(sentence_from_doc.words))])
            if o_word not in flattened_string or t_word not in flattened_string:
                continue
            
            try: pattern_counter['-'.join([dep_rel for token, pos, dep_rel in d.print_(sentence_from_doc, o_word, t_word) if dep_rel != 'root'])] += 1
            except: 
                err_list.append([row['content'], o_word, t_word, row['opinion_words'], row['target']])
                raise KeyError(document)
err_df = pd.DataFrame(err_list, columns =['content', 'current_opinion_word', 'current_target_word', 'opinion_words', 'targets'])  

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8,

In [13]:
pattern_counter

defaultdict(int,
            {'advcl-advcl-nsubj': 1,
             'advcl-det-nsubj': 1,
             'advcl-fixed-advcl-obl-compound': 1,
             'advcl-nsubj-det-obl': 1,
             'advcl-xcomp-nsubj': 1,
             'advmod-parataxis-nsubj-compound': 1,
             'advmod-parataxis-obl-det-nsubj': 1,
             'amod': 1,
             'amod-compound-conj': 1,
             'amod-conj': 4,
             'amod-conj-obj-conj': 2,
             'amod-conj-obj-det-nsubj': 1,
             'amod-conj-punct-obj-appos-nmod-compound-compound': 1,
             'amod-nmod': 2,
             'amod-nmod-acl:relcl-xcomp': 1,
             'amod-nmod-acl:relcl-xcomp-conj': 1,
             'amod-nmod-nmod': 3,
             'amod-nmod-nmod-acl:relcl-xcomp': 2,
             'amod-nmod-nmod-acl:relcl-xcomp-conj': 2,
             'amod-nsubj': 1,
             'amod-obj': 1,
             'amod-obj-parataxis': 1,
             'amod-obl-det-nsubj': 2,
             'amod-obl-det-nsubj-nmod-nmod-comp

In [14]:
err_df

,content,current_opinion_word,current_target_word,opinion_words,targets
